# Euler–Maclaurin 2D formula

In [1]:
def num_int(f, a, b):
    from scipy import integrate

    try:
        integral = integrate.quad(f, a, b)

        result = integral[0]
        return result

    except Exception as ex:

        print ("Exception ex = ", str(ex))
        print ("f = ", f)
        try:
            #print ("integrand = ", get_integrand_view(f))
            print ("integrand = ", f(x))
        except Exception as ex2:
            print ("Exception ex2 = ", ex2)

        print ("a, b = ", a, ",", b)
        try:
            print ("f(a) = ", f(a))
        except:
            pass
        try:
            print ("f(b) = ", f(b))
        except:
            pass

        import inspect
        stack = inspect.stack()
        for frame in stack:
            func_name = frame[3]
            print ("func_name = ", func_name)
        
        raise ex

In [2]:
logging = False

calc_R_p=True

def my_euler_maclaurin1(f,symb,a,b,p, f_diff_symb_p=None):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        print("")


    sum_dfdx_a_bernoullis = sum([(f.diff(symb,k-1))*(bernoulli(k)/factorial(k)) for k in range(1,1+p)])
    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis.subs(symb == a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis.subs(symb == b))

    if True:
        B = lambda x, p : bernoulli_polynomial(x, p)
        P = lambda x, p : bernoulli_polynomial(x - floor(x), p)
        fact = lambda n : factorial(n)
    else:
        B = function("B")(x,p)
        P = function("P")(x,p)
        fact = function("fact")(n)
        
    if True == calc_R_p:
        # Can't calculate derivative wrt j + u.
        #int_R_p = lambda j : integral(f.diff(symb,p).subs(symb == u+j) * bernoulli_polynomial(u,p)/factorial(p), (u,0,1))
        def int_R_p(f,symb,j):
            v = var("v")            
            if None == f_diff_symb_p:
                deriv = f.diff(symb,p)
            else:
                deriv = f_diff_symb_p
            if logging:
                print("deriv", deriv)
            deriv_s = deriv.subs(symb == v+j)
            if logging:
                print("deriv_s", deriv_s)
            int_f_diff_B_fact = integral(deriv_s * bernoulli_polynomial(v,p)/factorial(p), (v,0,1))
            if logging:
                print("int_f_diff_B_fact", int_f_diff_B_fact)
            return int_f_diff_B_fact
        
        def int_R_p_shifted_first(f,symb,j):
            v = var("v")
            fn_shifted = f.subs(symb == v + j)
            if logging:
                print("fn_shifted", fn_shifted)
            deriv = fn_shifted.diff(v,p)
            if logging:
                print("deriv", deriv)
            int_f_diff_B_fact = integral(deriv * B(x=v,p=p)/fact(n=p), (v,0,1))
    
            if logging:
                print("int_f_diff_B_fact", int_f_diff_B_fact)

            return int_f_diff_B_fact

        if logging:
            print("int_R_p(j = a)", int_R_p(f,symb,j = a))
            
        if use_int_R_p_j:
            v, j = var("v, j")
            assume(j>0)
            from sage.calculus.calculus import symbolic_sum
            if change_R_p_order:
                # Change order of summation and integration
                if use_int_R_p_shifted_first:
                    R_p = (-1)^(p+1)*integral(symbolic_sum(f.subs(symb == v+j).diff(v,p)*B(x=v,p=p)/fact(n=p), \
                                                  j, a, b-1), \
                                              (v,0,1))
                else:
                    if None == f_diff_symb_p:
                        f_diff_symb_p = f.diff(symb,p)
                    R_p = (-1)^(p+1)*integral(symbolic_sum(f_diff_symb_p.subs(symb == v+j)*B(x=v,p=p)/fact(n=p), \
                                                  j, a, b-1), \
                                              (v,0,1))
            else:
                if use_int_R_p_shifted_first:
                    R_p = (-1)^(p+1)*symbolic_sum(int_R_p_shifted_first(f,symb,j=j), j, a, b-1)
                else:
                    R_p = (-1)^(+1)*symbolic_sum(int_R_p(f,symb,j=j), j, a, b-1)
                
        else:
            if None == f_diff_symb_p:
                f_diff_symb_p = f.diff(symb,p)
            if not use_num_int:
                R_p = (-1)^(p+1)*integral(f_diff_symb_p * P(x=symb,p=p)/fact(n=p),(symb,a,b),hold=True)
            else:
                R_p = (-1)^(p+1)*num_int(lambda symb : f_diff_symb_p * P(x=symb,p=p)/fact(n=p), a, b)
                

    int_fn_a_b = integral(f, (symb, a, b))
    if logging:
        print("int_fn_a_b", int_fn_a_b)
    
    s = int_fn_a_b - sum_dfdx_a_bernoullis.subs(symb == a)
    if Infinity != b:
        s += sum_dfdx_a_bernoullis.subs(symb == b)
    if True == calc_R_p:
        s += R_p
        
    if False:
        print("int_fn_a_b", int_fn_a_b)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis(a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis(b))
        if True == calc_R_p:
            print("int_R_p(j = a)", int_R_p(j = a))
            print("R_p", R_p)
        print("")
        print("s",s)
    return s

$${\displaystyle \sum _{i=a}^{b}f(i)=\int _{a}^{b}f(x)\,dx+{\frac {f(a)+f(b)}{2}}+\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{2k}}{(2k)!}}(f^{(2k-1)}(b)-f^{(2k-1)}(a))+R_{p},}$$

$${\displaystyle \sum _{i=a}^{b}f(i)=\int _{a}^{b}f(x)\,dx+\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(f^{(k-1)}(b)-f^{(k-1)}(a))+R_{p},}$$

$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a}^{b-1} \int _{0}^{1}f^{(p)}(v+j){\frac {B_{p}(v)}{p!}}\,dv.}$$

$${\displaystyle P_{k}(x)=B_{k}(x-\lfloor x\rfloor ),}$$

$${\displaystyle R_{p}=(-1)^{p+1}\int _{a}^{b}f^{(p)}(x){\frac {P_{p}(x)}{p!}}\,dx.}$$

In [3]:
# https://www.journals.vu.lt/LMJ/article/view/20600/19701
# ФОРМУЛА СУММИРОВАНИЯ ЭЙЛЕРА—МАКЛОРЕНА ДЛЯ ФУНКЦИИ МНОГИХ ПЕРЕМЕННЫХ А.БИКЯЛИС
# VIII LIETUVOS MATEMATIKOS RINKINYS 4
# ЛИТОВСКИЙ МАТЕМАТИЧЕСКИЙ СБОРНИК
# 1968

# Далее,обе стороны этого равенства суммируем по всем целым m2 из интервала(z2,y2). Получаем

if True:
    B = lambda x, p : bernoulli_polynomial(x, p)
    P = lambda x, p : bernoulli_polynomial(x - floor(x), p)
    fact = lambda n : factorial(n)
else:
    B = function("B")(x,p)
    P = function("P")(x,p)
    fact = function("fact")(n)

def integral_R_p(f,symb,jx,hold, f_diff_symb_p=None):
    vx = var("v_"+str(symb))
    if None == f_diff_symb_p:
        deriv = f.diff(symb,p)
    else:
        deriv = f_diff_symb_p
    if logging:
        print("deriv", deriv)

    deriv_subs = deriv.subs(symb == vx+jx)
    if logging:
        print("deriv_subs", deriv_subs)

    int_f_diff_B_fact = integral(deriv_subs * bernoulli_polynomial(vx,p)/factorial(p), (vx,0,1), hold=hold)
    if logging:
        print("int_f_diff_B_fact", int_f_diff_B_fact)

    return int_f_diff_B_fact

def integral_R_p_shifted_first(f,symb,jx,hold):
    vx = var("v_"+str(symb))
    fn_shifted = f.subs(symb == vx + jx)
    if logging:
        print("fn_shifted", fn_shifted)

    deriv = fn_shifted.diff(vx,p)
    if logging:
        print("deriv", deriv)

    int_f_diff_B_fact = integral(deriv * B(x=vx,p=p)/fact(n=p), (vx, 0, 1), hold=hold)

    if logging:
        print("int_f_diff_B_fact", int_f_diff_B_fact)

    return int_f_diff_B_fact
    
    
def euler_maclaurin_R_p(f,symb,a,b,p, f_diff_symb_p=None, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        if f_diff_symb_p is not None:
            print("f_diff_symb_p", f_diff_symb_p)
        print("")
    if use_int_R_p_j:
        jx = var('j_'+str(symb))
        assume(jx>0)
        from sage.calculus.calculus import symbolic_sum
        
        if change_R_p_order:
            vx = var("v_"+str(symb))
            # Change order of summation and integration
            if use_int_R_p_shifted_first:
                R_p = (-1)^(p+1)*integral(symbolic_sum(f.subs(symb == vx+jx).diff(vx,p)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a, b-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int)
            else:
                if None == f_diff_symb_p:
                    f_diff_symb_p = f.diff(symb,p)
                R_p = (-1)^(p+1)*integral(symbolic_sum(f_diff_symb_p.subs(symb == vx+jx)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a, b-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int)
        else:       
            if use_int_R_p_shifted_first:
                term = (-1)^(p+1)*integral_R_p_shifted_first(f=f, symb=symb, jx=jx, hold=hold_int)
                R_p = symbolic_sum(term, jx, a, b-1, hold=hold_sum)
            else:
                term = (-1)^(p+1)*integral_R_p(f=f, symb=symb, jx=jx, hold=hold_int, f_diff_symb_p=f_diff_symb_p)
                R_p = symbolic_sum(term, jx, a, b-1, hold=hold_sum)
            return R_p, term
    else:
        if not use_num_int:
            if None == f_diff_symb_p:
                f_diff_symb_p = f.diff(symb,p)
            if logging:
                print("(-1)^(p+1)*P(x=symb,p=p)/fact(n=p)", (-1)^(p+1)*P(x=symb,p=p)/fact(n=p))
            R_p = (-1)^(p+1)*integral(f_diff_symb_p * P(x=symb,p=p)/fact(n=p),(symb,a,b),hold=hold_int)
        else:
            R_p = (-1)^(p+1)*num_int(lambda symb: f_diff_symb_p * P(x=symb,p=p)/fact(n=p), a, b)
            
    return R_p

$$\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(f^{(k-1)}(b)-f^{(k-1)}(a))$$

In [4]:
def sum_dfdx_bernoulis(f,symb,a,b,p):
    dfdx_a_bernoullis = []
    for k in range(1,1+p):
        dfdx_a_bernoullis += [(f.diff(symb,k-1))*(bernoulli(k)/factorial(k))]
        
    sum_dfdx_a_bernoullis = sum(dfdx_a_bernoullis)

    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis.subs(symb == a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis.subs(symb == b))

    s = - sum_dfdx_a_bernoullis.subs(symb == a)
    if Infinity != b:
        s += sum_dfdx_a_bernoullis.subs(symb == b)
    return s


In [5]:
def sum_dfdx_bernoulis_str(f,symb,a,b,p):
    sum_dfdx_a_bernoullis = ""
    for k in range(1,1+p):
        if k - 1 > 0:
            sum_dfdx_a_bernoullis += " + (" + str(bernoulli(k)/factorial(k)) + ") * (" + str(f) + ").diff(" + str(symb) + ", " + str(k-1) + ")"
        else:
            sum_dfdx_a_bernoullis += " + (" + str(bernoulli(k)/factorial(k)) + ") * (" + str(f) + ")"
            

    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", "(" + sum_dfdx_a_bernoullis + ").subs(" + str(symb) + "==" + str(a) + ")")
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", "(" + sum_dfdx_a_bernoullis + ").subs(" + str(symb) + "==" + str(b) + ")")
    s = ""

    s += "- (" + sum_dfdx_a_bernoullis + ").subs(" + str(symb) + "==" + str(a) + ")"
    if Infinity != b:
        s += " + (" + sum_dfdx_a_bernoullis + ").subs(" + str(symb) + "==" + str(b) + ")"
    return s


In [6]:
# sumy_sumx = sum_dfdx_bernoulis(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def sum_dfdy_bernoulis_sum_dfdx_bernoulis_str(f, x, a_x, b_x, y, a_y, b_y, p):
    str_f = sum_dfdx_bernoulis_str(f(x,y), x, a_x, b_x, p)
    sum_dfdx_a_bernoullis = ""
    for k in range(1,1+p):
        if k - 1 > 0:
            sum_dfdx_a_bernoullis += " + (" + str(bernoulli(k)/factorial(k)) + ") * (" + str_f + ").diff(" + str(y) + ", " + str(k-1) + ")"
        else:
            sum_dfdx_a_bernoullis += " + (" + str(bernoulli(k)/factorial(k)) + ") * (" + str_f + ")"

    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", "(" + sum_dfdx_a_bernoullis + ").subs(" + str(y) + "==" + str(a_y) + ")")
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", "(" + sum_dfdx_a_bernoullis + ").subs(" + str(y) + "==" + str(b_y) + ")")
    s = ""

    s += "- (" + sum_dfdx_a_bernoullis + ").subs(" + str(y) + "==" + str(a_y) + ")"
    if Infinity != b_y:
        s += " + (" + sum_dfdx_a_bernoullis + ").subs(" + str(y) + "==" + str(b_y) + ")"
    return s

In [7]:
def definite_integral_derivative(f, x, a, b, diff_param, hold=True):
    from sage.symbolic.integration.integral import definite_integral

    if not x.has(diff_param):
        # integration variable != differentiation variable
        ans = definite_integral(f.diff(diff_param), x, a, b, hold=hold)
    else:
        ans = SR.zero()

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += f.subs(x == b) * b.diff(diff_param)
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans -= f.subs(x == a) * a.diff(diff_param)

    return ans


In [8]:
def definite_integral_derivative_str(f, x, a, b, diff_param=None, hold=False, algorithm=None):

    if not x.has(diff_param):
        # integration variable != differentiation variable
        if algorithm is not None:
            ans = "integrate((" + str(f) + ").diff(" + str(diff_param) + "), " + str(x) + ", " + str(a) + ", " + str(b) + ", algorithm = "\""+ algorithm + ", hold = " + str(hold) + ")"
        else:
            ans = "definite_integral((" + str(f) + ").diff(" + str(diff_param) + "), " + str(x) + ", " + str(a) + ", " + str(b) + ", hold = " + str(hold) + ")"
    else:
        ans = ""

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += " + " + str(f.subs(x == b) * b.diff(diff_param))
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans += " - " + str(f.subs(x == a) * a.diff(diff_param))

    return ans

In [9]:
def definite_num_int_derivative_str(f, x, a, b, diff_param=None):

    if not x.has(diff_param):
        # integration variable != differentiation variable
        ans = "num_int(lambda " + str(x)+ " : " + str(f.diff(diff_param)) + ", " + str(a) + ", " + str(b) + ")"
    else:
        ans = ""

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += " + " + str(f.subs(x == b) * b.diff(diff_param))
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans += " - " + str(f.subs(x == a) * a.diff(diff_param))

    return ans

$$\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(g(y)^{(k-1)}(b_y)-g(y)^{(k-1)}(a_y))$$
where 

$$g(y) = {\displaystyle R_{p}(y)=(-1)^{p+1}\sum_{j=a_x}^{b_x-1} \int _{0}^{1}\frac{\partial^{p}}{\partial x^{p}}f(v_x+j,y){\frac {B_{p}(v_x)}{p!}}\,dv_x.}$$
or

$${\displaystyle P_{k}(x)=B_{k}(x-\lfloor x\rfloor ),}$$

$$g(y) = {\displaystyle R_{p}=(-1)^{p+1}\int _{a_x}^{b_x}\frac{\partial^{p}}{\partial x^{p}}f(x,y){\frac {P_{p}(x)}{p!}}\,dx.}$$

In [10]:
#int_R_p = integral(euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p), (y, a_y, b_y), hold=True)

def integral_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p, f_diff_x_p, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("f(x,y)", f(x,y))
        if None != f_diff_x_p:
            print("f_diff_x_p", f_diff_x_p)
            print("f_diff_x_p(x,y)", f_diff_x_p(x,y))
        print("")
    if use_int_R_p_j:
        jx = var('j_x')
        assume(jx>0)
        from sage.calculus.calculus import symbolic_sum
        
        if change_R_p_order:
            vx = var("v_"+str(x))
            # Change order of summation and integration
            if use_int_R_p_shifted_first:
                int_R_p = (-1)^(p+1)*integral(integral(symbolic_sum(f.subs(x == vx+jx).diff(vx,p)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a_x, b_x-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int), y, a_y, b_y, hold=hold_int)
            else:
                if None == f_diff_x_p:
                    f_diff_x_p = f.diff(x,p)
                int_R_p = (-1)^(p+1)*integral(integral(symbolic_sum(f_diff_x_p.subs(x == vx+jx)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a_x, b_x-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int), y, a_y, b_y, hold=hold_int)
        else:       
            if use_int_R_p_shifted_first:
                int_int = (-1)^(p+1)*integral(integral_R_p_shifted_first(f=f, symb=x, jx=jx, hold=hold_int), y, a_y, b_y, hold=hold_int)
                int_R_p = symbolic_sum(int_int, jx, a_x, b_x-1, hold=hold_sum)
            else:
                int_int = (-1)^(p+1)*integral(integral_R_p(f=f, symb=x, jx=jx, hold=hold_int), y, a_y, b_y, hold=hold_int)
                int_R_p = symbolic_sum(int_int, jx, a_x, b_x-1, hold=hold_sum)
            return (int_R_p, int_int)
    else:
        if not use_num_int:
            if None == f_diff_x_p:
                f_diff_x_p = f.diff(x,p)
            int_R_p = (-1)^(p+1)*integral(integral(f_diff_x_p * P(x=x,p=p)/fact(n=p),(x,a_x,b_x),hold=hold_int), y, a_y, b_y, hold=hold_int)
        else:
            int_R_p = (-1)^(p+1)*num_int(lambda y : num_int(lambda x : f_diff_x_p(x,y) * P(x=x,p=p)/fact(n=p), a_x, b_x), a_y, b_y)
            
    return int_R_p

In [11]:
#sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def sum_dfdx_bernoulis_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p):
    if logging:
        print("sum_dfdx_bernoulis_euler_maclaurin_R_p", f, x, a_x, b_x, y, a_y, b_y, p)
    dfdx_a_bernoullis = []
    dfdx_a_bernoullis_terms = []
    from sage.calculus.calculus import symbolic_sum
    from sage.symbolic.integration.integral import definite_integral
    
    for k in range(1,1+p):
        if 1 == k:
            if use_int_R_p_j:
                vx, jx = var("v_x, j_x")
                term_1 = lambda y_symb, y_val, p, jx : (-1)^(p+1)*definite_integral(f(x,y_symb).diff(x,p) \
                                                                                 .subs(x == vx + jx) \
                                                                      * B(x=vx,p=p)/fact(n=p),vx,0,1,hold=False) \
                                                                                .subs(y_symb == y_val)
                if logging:
                    print("k", k)
                    print("term_1(y_symb=y, y_val=a_y,jx=a_x) =", term_1(y_symb=y, y_val=a_y,jx=a_x))
                    
                R_p_1 = lambda y_symb, y_val, p : symbolic_sum(expression=term_1(y_symb=y_symb, y_val=y_val, p=p, jx=jx), \
                                                                     v=jx, a=a_x, b=b_x-1)
                dfdx_a_bernoullis_terms += [lambda y_symb, y_val, p, jx, i=i : \
                                            term_1(y_symb=y_symb, y_val=y_val, jx=jx)*(bernoulli(k)/factorial(k))]
            else:
                R_p_1 = lambda y_symb, y_val, p : (-1)^(p+1)*definite_integral(f(x,y_symb).diff(x,p) * P(x=x,p=p)/fact(n=p), \
                                                                  x, a_x, b_x, hold=True) \
                                                                  .subs(y_symb == y_val)
                if logging:
                    print("k", k)
                    print("R_p_1(y_symb=y, y_val=a_y) =", R_p_1(y, y, p))
                    print("R_p_1(y_symb=y, y_val=a_y) =", R_p_1(y_symb=y, y_val=a_y, p=p))
                
            if logging:
                print("R_p(y_symb=y, y_val=a_y) =", R_p_1(y_symb=y, y_val=a_y, p=p))
                print("\n")

            dfdx_a_bernoullis += [lambda y_symb, y_val, p, i=i : R_p_1(y_symb=y_symb, y_val=y_val, p=p)*(bernoulli(k)/factorial(k))]
            if logging:
                print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
                print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))

                for ii in range(len(dfdx_a_bernoullis)):
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii])
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii](y, y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y, a_y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y_symb=y, y_val=a_y, p=p))

        elif 2 == k:
            if use_int_R_p_j:
                vx, jx = var("v_x, j_x")
                term_2 = lambda y_symb, y_val, p, jx : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                                                                 .subs(x == vx + jx) \
                                        * B(x=vx,p=p)/fact(n=p), x=vx, a=0, b=1, diff_param=y_symb, hold=False) \
                                                                                .subs(y_symb == y_val)

                if logging:
                    print("k", k)
                    print("term_2(y_symb=y, y_val=a_y,jx=a_x) =", term_2(y_symb=y, y_val=a_y, p=p, jx=a_x))

                R_p_2 = lambda y_symb, y_val, p : symbolic_sum(expression=term_2(y_symb=y_symb, y_val=y_val, p=p, jx=jx), \
                                                                 v=jx, a=a_x, b=b_x-1)
                dfdx_a_bernoullis_terms += [lambda y_symb, y_val, p, jx, i=i : \
                                            term_2(y_symb=y_symb, y_val=y_val, p=p, jx=jx)*(bernoulli(k)/factorial(k))]
            else:
                R_p_2 = lambda y_symb, y_val, p : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                            * P(x=x,p=p)/fact(n=p), x=x, a=a_x, b=b_x, diff_param=y_symb) \
                                                  .subs(y_symb == y_val)

                if logging:
                    print("k", k)
                    print("R_p_2(y_symb=y, y_val=a_y) =", R_p_2(y, y, p))
                    print("R_p_2(y_symb=y, y_val=a_y) =", R_p_2(y_symb=y, y_val=a_y, p=p))
                
            if logging:
                print("R_p(y_symb=y, y_val=a_y) =", R_p_2(y_symb=y, y_val=a_y, p=p))
                print("\n")
        
            dfdx_a_bernoullis += [lambda y_symb, y_val, p, i=i : R_p_2(y_symb=y_symb, y_val=y_val, p=p)*(bernoulli(k)/factorial(k))]
            if logging:
                print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
                print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))

                for ii in range(len(dfdx_a_bernoullis)):
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii])
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii](y, y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y, a_y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y_symb=y, y_val=a_y, p=p))
        else:
            if use_int_R_p_j:
                vx, jx = var("v_x, j_x")
                term_3 = lambda y_symb, y_val, p, jx : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                                                                 .subs(x == vx + jx) \
                                        * B(x=vx,p=p)/fact(n=p), x=vx, a=0, b=1, diff_param=y_symb, hold=False) \
                                            .diff(y_symb, k-2) .subs(y_symb == y_val)

                if logging:
                    print("k", k)
                    print("term_3(y_symb=y, y_val=a_y,jx=a_x) =", term_3(y_symb=y, y_val=a_y, p=p, jx=a_x))
                    
                R_p_3 = lambda y_symb, y_val, p : symbolic_sum(expression=term_3(y_symb=y_symb, y_val=y_val, p=p, jx=jx), \
                                                                 v=jx, a=a_x, b=b_x-1)
                dfdx_a_bernoullis_terms += [lambda y_symb, y_val, p, jx, i=i : \
                                            term_3(y_symb=y_symb, y_val=y_val, p=p, jx=jx)*(bernoulli(k)/factorial(k))]
            else:
                R_p_3 = lambda y_symb, y_val, p : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                                    * P(x=x,p=p)/fact(n=p), x=x, a=a_x, b=b_x, diff_param=y_symb) \
                                                    .diff(y_symb, k-2).subs(y_symb == y_val)
                
            if logging:
                print("R_p(y_symb=y, y_val=a_y) =", R_p_3(y_symb=y, y_val=a_y, p=p))
                print("\n")

            dfdx_a_bernoullis += [lambda y_symb, y_val, p, i=i : R_p_3(y_symb=y_symb, y_val=y_val, p=p)*(bernoulli(k)/factorial(k))]
            if logging:
                print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
                print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))

                for ii in range(len(dfdx_a_bernoullis)):
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii])
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii](y, y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y, a_y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y_symb=y, y_val=a_y, p=p))

    if logging:
        print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
        print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))
        
        for ii in range(len(dfdx_a_bernoullis)):
            print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii])
            print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii](y, y, p))
            print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y, a_y, p))
            print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y_symb=y, y_val=a_y, p=p))



    sum_dfdx_a_bernoullis = lambda y_symb, y_val, p : sum([dfdx_a_bernoullis[i](y_symb, y_val, p) \
                                                      for i in range(len(dfdx_a_bernoullis))])
    
    sum_dfdx_a_bernoullis_terms = lambda y_symb, y_val, p, jx : sum([dfdx_a_bernoullis_terms[i](y_symb, y_val, p, jx) \
                                                               for i in range(len(dfdx_a_bernoullis_terms))])
    
    if logging:
        print("sum_dfdx_a_bernoullis =", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a_y) =", sum_dfdx_a_bernoullis(y_symb=y, y_val=a_y, p=p))
    
    s = - sum_dfdx_a_bernoullis(y_symb=y, y_val=a_y, p=p)
    if b_y is Infinity:
        s_terms = lambda jx : - sum_dfdx_a_bernoullis_terms(y_symb=y, y_val=a_y, p=p, jx=jx)
    else:
        s += sum_dfdx_a_bernoullis(y_symb=y, y_val=b_y, p=p)
        s_terms = lambda jx : sum_dfdx_a_bernoullis_terms(y_symb=y, y_val=b_y, p=p, jx=jx) - sum_dfdx_a_bernoullis_terms(y_symb=y, y_val=a_y, p=p, jx=jx)
    return s, s_terms

$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a_y}^{b_y-1} \int _{0}^{1}g^{(p)}(v_y+j){\frac {B_{p}(v_y)}{p!}}\,dv_y.}$$

$${\displaystyle P_{k}(y)=B_{k}(y-\lfloor y\rfloor ),}$$

$${\displaystyle R_{p}=(-1)^{p+1}\int _{a_y}^{b_y}g^{(p)}(y){\frac {P_{p}(y)}{p!}}\,dy.}$$

where

$$g(y) = {\displaystyle R_{p}(y)=(-1)^{p+1}\sum_{j=a_x}^{b_x-1} \int _{0}^{1}\frac{\partial^{p}}{\partial x^{p}}f(v_x+j,y){\frac {B_{p}(v_x)}{p!}}\,dv_x.}$$
or

$${\displaystyle P_{k}(x)=B_{k}(x-\lfloor x\rfloor ),}$$

$$g(y) = {\displaystyle R_{p}=(-1)^{p+1}\int _{a_x}^{b_x}\frac{\partial^{p}}{\partial x^{p}}f(x,y){\frac {P_{p}(x)}{p!}}\,dx.}$$

In [12]:
#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def euler_maclaurin_R_p_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p, f_diff_x_p_diff_y_p):

    from sage.calculus.calculus import symbolic_sum
    from sage.symbolic.integration.integral import definite_integral
    if use_int_R_p_j:
        vx,vy = var("v_x, v_y")
        jx,jy = var("j_x, j_y")
        assume(v_y+j_y>0)
        
        if None == f_diff_x_p_diff_y_p:
            f_diff_x_p_diff_y_p = f.diff(x,p).diff(y,p)

        R_p_R_p = (-1)^(p+1)*symbolic_sum(definite_integral( \
                               (-1)^(p+1)*symbolic_sum(definite_integral(f_diff_x_p_diff_y_p.subs(x == vx + jx)  \
                                                              * B(x=vx,p=p)/fact(n=p), \
                                 vx,0,1,hold=True), v=jx, a=a_x, b=b_x-1,hold=True) \
                                                                        .diff(y,p).subs(y == vy + jy)  \
                                                              * B(x=vy,p=p)/fact(n=p), \
                                 vy,0,1,hold=True), v=jy, a=a_y, b=b_y-1,hold=True)
    else:
        if not use_num_int:

            if None == f_diff_x_p_diff_y_p:
                f_diff_x_p_diff_y_p = f.diff(x,p)

            R_p_R_p = (-1)^(p+1)*definite_integral( \
                               (-1)^(p+1)*definite_integral(f_diff_x_p_diff_y_p  \
                                                              * P(x=x,p=p)/fact(n=p), \
                                 x, a_x, b_x, hold=True).diff(y,p)  \
                                                              * P(x=y,p=p)/fact(n=p), \
                                 y, a_y, b_y, hold=True)
        else:
            f_diff_x_p_diff_y_p_num_str = "f_diff_x_diff_y_num = lambda " + str(x) + ", " + str(y) + ":" + str(f(x,y).diff(x,p).diff(y,p))
            print("f_diff_x_p_diff_y_p_num_str=", f_diff_x_p_diff_y_p_num_str)                                                                          
            exec(preparse(f_diff_x_p_diff_y_p_num_str))
            R_p_R_p = (-1)^(p+1)*num_int(lambda y : \
                               (-1)^(p+1)*num_int(lambda x : f_diff_x_p_diff_y_p_num(x,y)  \
                                                              * P(x=x,p=p)/fact(n=p), \
                                 a_x, b_x)  \
                                                              * P(x=y,p=p)/fact(n=p), \
                                 a_y, b_y)
        
    return R_p_R_p

In [13]:
#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def euler_maclaurin_R_p_euler_maclaurin_R_p_num_int(f_diff_x_p_diff_y_p, x, a_x, b_x, y, a_y, b_y, p):
    R_p_R_p = (-1)^(p+1)*num_int(lambda y : \
                       (-1)^(p+1)*num_int(lambda x : f_diff_x_p_diff_y_p(x,y)  \
                                                      * P(x=x,p=p)/fact(n=p), \
                         a_x, b_x)  \
                                                      * P(x=y,p=p)/fact(n=p), \
                         a_y, b_y)
    return R_p_R_p

In [14]:
def euler_maclaurin1(f, symb, a, b, p, f_diff_symb_p=None, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        print("")
    #u, j = var("u, j")

    int_fn_a_b = integral(f, (symb, a, b))
    if logging:
        print("int_fn_a_b", int_fn_a_b)
    
    #s = int_fn_a_b + sum_dfdx_a_bernoullis(b) - sum_dfdx_a_bernoullis(a) + R_p
    s = int_fn_a_b + sum_dfdx_bernoulis(f,symb,a,b,p)
    
    if True == calc_R_p:
        R_p = euler_maclaurin_R_p(f, symb, a, b, p, f_diff_symb_p=f_diff_symb_p,hold_int=hold_int, hold_sum=hold_sum)
        if type(R_p) is tuple:
            s += R_p[0]
        else:
            s += R_p

    return s

In [15]:
from IPython.display import display, Math, Latex

In [16]:
a_x = var("n_x")
b_x = var("b_x")
a_y = var("n_y")
b_y = var("b_y")

assume(a_x, 'real')
assume(b_x, 'real')
assume(a_y, 'real')
assume(b_y, 'real')

assume(a_x, 'integer')
assume(b_x, 'integer')
assume(a_y, 'integer')
assume(b_y, 'integer')

assume(a_x > 0)
assume(b_x > 0)
assume(a_y > 0)
assume(b_y > 0)

x,y = var("x,y")

n = QQ['n'].0
j = QQ['j'].0

p = 4
p = 3
#p = 2
#p = 1

F = function("F")(x, y)
print(F)

s [_SAGE_VAR_n_x>0]
parser_output [n_x > 0]
s [_SAGE_VAR_b_x>0]
parser_output [b_x > 0]
s [_SAGE_VAR_n_y>0]
parser_output [n_y > 0]
s [_SAGE_VAR_b_y>0]
parser_output [b_y > 0]
F(x, y)


$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a}^{b-1} \int _{0}^{1}f^{(p)}(u+j){\frac {B_{p}(u)}{p!}}\,du.}$$

In [17]:
use_int_R_p_j=True
change_R_p_order=True
use_int_R_p_shifted_first=True
use_num_int = False

In [18]:
inty = integral           (F(x,y),(y, a_y, b_y))
print("inty=", inty)
#print("inty=$", latex(inty),"$")
display(Math(latex(inty)))

sumy = sum_dfdx_bernoulis (F(x,y), y, a_y, b_y,p)
print("sumy=", sumy)
#print("sumy=$", latex(sumy),"$")
display(Math(latex(sumy)))

sumy = sum_dfdx_bernoulis_str (F(x,y), y, a_y, b_y,p)
print("sumy=", sumy)
exec(preparse("sum_y=" + sumy))
print(preparse("sum_y=" + sumy))
#print("sum_y=$", latex(sum_y),"$")
display(Math("sum_y=" + latex(sum_y)))

R_py = euler_maclaurin_R_p(F(x,y), y, a_y, b_y,p)
print("R_py=", R_py)
#print("R_py=$", latex(R_py),"$")
display(Math(latex(R_py)))

inty= integrate(F(x, y), y, n_y, b_y)


<IPython.core.display.Math object>

sumy= 1/2*F(x, n_y) - 1/12*diff(F(x, n_y), n_y)


<IPython.core.display.Math object>

sumy= - ( + (-1/2) * (F(x, y)) + (1/12) * (F(x, y)).diff(y, 1) + (0) * (F(x, y)).diff(y, 2)).subs(y==n_y)
sum_y=- ( + (-Integer(1)/Integer(2)) * (F(x, y)) + (Integer(1)/Integer(12)) * (F(x, y)).diff(y, Integer(1)) + (Integer(0)) * (F(x, y)).diff(y, Integer(2))).subs(y==n_y)


<ipython-input-18-74d9e814f2c2>:13: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  exec(preparse("sum_y=" + sumy))


<IPython.core.display.Math object>

s [_SAGE_VAR_j_y>0]
parser_output [j_y > 0]
s _SAGE_VAR_n_y-_SAGE_VAR_j_y
parser_output -j_y + n_y
s _SAGE_VAR_n_y-_SAGE_VAR_j_y
parser_output -j_y + n_y
s _SAGE_VAR_n_y-_SAGE_VAR_j_y
parser_output -j_y + n_y
s _SAGE_VAR_n_y-_SAGE_VAR_j_y
parser_output -j_y + n_y
s _SAGE_VAR_b_y-_SAGE_VAR_j_y
parser_output b_y - j_y
s _SAGE_VAR_b_y-_SAGE_VAR_j_y
parser_output b_y - j_y
s _SAGE_VAR_b_y-_SAGE_VAR_j_y
parser_output b_y - j_y
s _SAGE_VAR_b_y-_SAGE_VAR_j_y
parser_output b_y - j_y
R_py= integrate(sum(1/12*(2*v_y^3 - 3*v_y^2 + v_y)*D[1, 1, 1](F)(x, j_y + v_y), j_y, n_y, b_y - 1), v_y, 0, 1)


<IPython.core.display.Math object>

In [19]:
int_int = integral(integral           (F(x,y),(x, a_x, b_x)),  (y, a_y, b_y))
print("int_int=", int_int)
#print("int_int=$", latex(int_int),"$")
display(Math(latex(int_int)))

int_sum = integral(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), (y, a_y, b_y))
print("int_sum=", int_sum)
#print("int_sum=$", latex(int_sum.expand()),"$")
display(Math(latex(int_sum.expand())))

sumx_str = sum_dfdx_bernoulis_str (F(x,y), x, a_x, b_x, p)
inty_sumx_str = "integral(" + sumx_str + ", (y, a_y, b_y))"
print("inty_sumx_str=", inty_sumx_str)
exec(preparse("inty_sumx=" + inty_sumx_str))
print(preparse("inty_sumx=" + inty_sumx_str))
print("inty_sumx = $", latex(inty_sumx),"$")
display(Math("inty\_sumx = " + latex(inty_sumx)))

R_px = euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p)
if type(R_px) is tuple:
    inty_R_px = integral(R_px[0], (y, a_y, b_y), hold=True)
else:
    inty_R_px = integral(R_px, (y, a_y, b_y), hold=True)

print("inty_R_px=",inty_R_px)
#print("inty_R_px=$",latex(inty_R_px),"$")
display(Math(latex(inty_R_px)))

inty_R_px = integral_euler_maclaurin_R_p(F(x,y), x, a_x, b_x, y, a_y, b_y, p, diff(F(x,y),x,p), hold_int=True, hold_sum=True)
print("inty_R_px=",inty_R_px)
#print("inty_R_px=$",latex(inty_R_px),"$")
display(Math(latex(inty_R_px)))


<>:17: DeprecationWarning: invalid escape sequence \_
<>:17: DeprecationWarning: invalid escape sequence \_
<ipython-input-19-0baf85e035c4>:17: DeprecationWarning: invalid escape sequence \_
  display(Math("inty\_sumx = " + latex(inty_sumx)))


int_int= integrate(integrate(F(x, y), x, n_x, b_x), y, n_y, b_y)


<IPython.core.display.Math object>

int_sum= -1/12*integrate(-6*F(n_x, y), y, n_y, b_y) - 1/12*integrate(diff(F(n_x, y), n_x), y, n_y, b_y)


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

s [_SAGE_VAR_j_x>0]
parser_output [j_x > 0]
s _SAGE_VAR_n_x-_SAGE_VAR_j_x
parser_output -j_x + n_x
s _SAGE_VAR_n_x-_SAGE_VAR_j_x
parser_output -j_x + n_x
s _SAGE_VAR_n_x-_SAGE_VAR_j_x
parser_output -j_x + n_x
s _SAGE_VAR_n_x-_SAGE_VAR_j_x
parser_output -j_x + n_x
s _SAGE_VAR_b_x-_SAGE_VAR_j_x
parser_output b_x - j_x
s _SAGE_VAR_b_x-_SAGE_VAR_j_x
parser_output b_x - j_x
s _SAGE_VAR_b_x-_SAGE_VAR_j_x
parser_output b_x - j_x
s _SAGE_VAR_b_x-_SAGE_VAR_j_x
parser_output b_x - j_x
inty_R_px= integrate(integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, y), j_x, n_x, b_x - 1), v_x, 0, 1), y, n_y, b_y)


<IPython.core.display.Math object>

s _SAGE_VAR_n_x-_SAGE_VAR_j_x
parser_output -j_x + n_x
s _SAGE_VAR_n_x-_SAGE_VAR_j_x
parser_output -j_x + n_x
s _SAGE_VAR_n_x-_SAGE_VAR_j_x
parser_output -j_x + n_x
s _SAGE_VAR_n_x-_SAGE_VAR_j_x
parser_output -j_x + n_x
s _SAGE_VAR_b_x-_SAGE_VAR_j_x
parser_output b_x - j_x
s _SAGE_VAR_b_x-_SAGE_VAR_j_x
parser_output b_x - j_x
s _SAGE_VAR_b_x-_SAGE_VAR_j_x
parser_output b_x - j_x
s _SAGE_VAR_b_x-_SAGE_VAR_j_x
parser_output b_x - j_x
inty_R_px= integrate(integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, y), j_x, n_x, b_x - 1), v_x, 0, 1), y, n_y, b_y)


<IPython.core.display.Math object>

In [20]:
sum_int = sum_dfdx_bernoulis(integral           (F(x,y),(x, a_x, b_x)),  y, a_y, b_y, p)
print("sum_int=",sum_int)
#print("sum_int=$",latex(sum_int),"$")
display(Math(latex(sum_int)))

sum_sum = sum_dfdx_bernoulis(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
print("sum_sum=",sum_sum)
#print("sum_sum=$",latex(sum_sum),"$")
display(Math(latex(sum_sum)))

use_int_R_p_j=False
if not use_int_R_p_j:
    R_px = euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p, f_diff_symb_p=F(x,y).diff(x,p))
    print("R_px=", R_px)
    #print("R_px=$",latex(R_px),"$")
    display(Math(latex(R_px)))

    sumy_R_px = sum_dfdx_bernoulis(R_px, y, a_y, b_y, p)
    print("sumy_R_px=",sumy_R_px)
    #print("sumy_R_px=$",latex(sumy_R_px),"$")
    display(Math(latex(sumy_R_px)))

    sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p, f_diff_symb_p=F(x,y).diff(x,p)), y, a_y, b_y, p)
    print("sum_R_p=",sum_R_p)
    #print("sum_R_p=$",latex(sum_R_p),"$")
    display(Math(latex(sum_R_p)))
else:
    # Sagemath 9.3 has bug diff of symbolic_sum
    # https://trac.sagemath.org/ticket/32161
    pass


sum_int= 1/2*integrate(F(x, n_y), x, n_x, b_x) - 1/12*integrate(diff(F(x, n_y), n_y), x, n_x, b_x)


<IPython.core.display.Math object>

sum_sum= 1/4*F(n_x, n_y) - 1/24*diff(F(n_x, n_y), n_x) + 1/144*diff(F(n_x, n_y), n_x, n_y) - 1/24*diff(F(n_x, n_y), n_y)


<IPython.core.display.Math object>

R_px= integrate(1/12*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, y), x, x, x), x, n_x, b_x)


<IPython.core.display.Math object>

sumy_R_px= 1/24*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x), x, n_x, b_x) - 1/144*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x, n_y), x, n_x, b_x)


<IPython.core.display.Math object>

sum_R_p= 1/24*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x), x, n_x, b_x) - 1/144*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x, n_y), x, n_x, b_x)


<IPython.core.display.Math object>

In [21]:
# with p = 3 I have
# bug with zeroing of array of lambdas
# here (inside sum_dfdx_bernoulis_euler_maclaurin_R_p)
use_int_R_p_j=False
logging = True
sum_R_p = sum_dfdx_bernoulis_euler_maclaurin_R_p(F(x,y), x, a_x, b_x,    y, a_y, b_y, p)

if type(sum_R_p) is tuple:
    print("sum_R_p=",sum_R_p[0])
    #print("sum_R_p=$",latex(sum_R_p[0]),"$")
    display(Math(latex(sum_R_p[0])))
else:
    print("sum_R_p=",sum_R_p)
    #print("sum_R_p=$",latex(sum_R_p),"$")
    display(Math(latex(sum_R_p)))

sum_dfdx_bernoulis_euler_maclaurin_R_p F(x, y) x n_x b_x y n_y b_y 3
k 1
R_p_1(y_symb=y, y_val=a_y) = integrate(1/12*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, y), x, x, x), x, n_x, b_x)
R_p_1(y_symb=y, y_val=a_y) = 1/12*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x), x, n_x, b_x)
R_p(y_symb=y, y_val=a_y) = 1/12*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x), x, n_x, b_x)


dfdx_a_bernoullis = [<function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a0567ee0>]
len(dfdx_a_bernoullis) = 1
dfdx_a_bernoullis [ 0 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a0567ee0>
dfdx_a_bernoullis [ 0 ]()= -1/2*integrate(1/12*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, y), x, x, x), x, n_x, b_x)
dfdx_a_bernoullis [ 0 ](a_y)= -1/24*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x

<ipython-input-11-4112e1ee9c8c>:25: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  print("R_p_1(y_symb=y, y_val=a_y) =", R_p_1(y, y, p))
<ipython-input-11-4112e1ee9c8c>:26: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  print("R_p_1(y_symb=y, y_val=a_y) =", R_p_1(y_symb=y, y_val=a_y, p=p))
<ipython-input-11-4112e1ee9c8c>:29: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details

R_p(y_symb=y, y_val=a_y) = 1/12*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x, n_y, n_y), x, n_x, b_x)


dfdx_a_bernoullis = [<function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a0567ee0>, <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a05671f0>, <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a077dc10>]
len(dfdx_a_bernoullis) = 3
dfdx_a_bernoullis [ 0 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a0567ee0>
dfdx_a_bernoullis [ 0 ]()= 0
dfdx_a_bernoullis [ 0 ](a_y)= 0
dfdx_a_bernoullis [ 0 ](a_y)= 0
dfdx_a_bernoullis [ 1 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a05671f0>
dfdx_a_bernoullis [ 1 ]()= 0
dfdx_a_bernoullis [ 1 ](a_y)= 0
dfdx_a_bernoullis [ 1 ](a_y)= 0
dfdx_a_bernoullis [ 2 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a077dc10>
dfdx_a_bernoulli

<ipython-input-11-4112e1ee9c8c>:94: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  dfdx_a_bernoullis += [lambda y_symb, y_val, p, i=i : R_p_3(y_symb=y_symb, y_val=y_val, p=p)*(bernoulli(k)/factorial(k))]


dfdx_a_bernoullis [ 2 ](a_y)= 0
dfdx_a_bernoullis [ 2 ](a_y)= 0
dfdx_a_bernoullis = [<function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a0567ee0>, <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a05671f0>, <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a077dc10>]
len(dfdx_a_bernoullis) = 3
dfdx_a_bernoullis [ 0 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a0567ee0>
dfdx_a_bernoullis [ 0 ]()= 0
dfdx_a_bernoullis [ 0 ](a_y)= 0
dfdx_a_bernoullis [ 0 ](a_y)= 0
dfdx_a_bernoullis [ 1 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a05671f0>
dfdx_a_bernoullis [ 1 ]()= 0
dfdx_a_bernoullis [ 1 ](a_y)= 0
dfdx_a_bernoullis [ 1 ](a_y)= 0
dfdx_a_bernoullis [ 2 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fc8a077dc10>
dfdx_a_bernoullis [ 2 ]()= 0
dfdx_a_bernoullis [ 2 ](a_y)= 0
dfdx_a_bernoullis [ 2 ](a_y)= 0
sum_dfdx_

<IPython.core.display.Math object>

In [22]:
R_p_int = euler_maclaurin_R_p(integral           (F(x,y),(x, a_x, b_x), hold=True), \
                              y, a_y, b_y, p,f_diff_symb_p=F(x,y).diff(y,p))
print("R_p_int=",R_p_int)
#print("R_p_int=$",latex(R_p_int),"$")
display(Math(latex(R_p_int)))

R_p_sum = euler_maclaurin_R_p(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), \
                              y, a_y, b_y, p, f_diff_symb_p=F(x,y).diff(y,p))
print("R_p_sum=",R_p_sum)
#print("R_p_sum=$",latex(R_p_sum),"$")
display(Math(latex(R_p_sum)))

try:
    if not use_int_R_p_j:
        R_px = euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p, f_diff_symb_p=F(x,y).diff(x,p))
        print("R_px=",R_px)
        #print("R_px=$",latex(R_px),"$")
        display(Math(latex(R_px)))

        R_px_diff_y_p = R_px.diff(y,p)
        print("R_px_diff_y_p=", R_px_diff_y_p)
        #print("R_px_diff_y_p=$", latex(R_px_diff_y_p),"$")
        display(Math(latex(R_px_diff_y_p)))

        logging = True

        R_p_R_p = euler_maclaurin_R_p(R_px, y, a_y, b_y, p, f_diff_symb_p=R_px_diff_y_p)

        logging = False

        print("R_p_R_p=",R_p_R_p)
        #print("R_p_R_p=$",latex(R_p_R_p),"$")
        display(Math(latex(R_p_R_p)))
    else:
        # Sagemath 9.3 has bug diff of symbolic_sum
        pass
except Exception as ex:
    print(ex)


try:
    R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p(F(x,y), x, a_x, b_x,    y, a_y, b_y, p, f_diff_x_p_diff_y_p=None)

    print("R_p_R_p=",R_p_R_p)
    #print("R_p_R_p=$",latex(R_p_R_p),"$")
    display(Math(latex(R_p_R_p)))
except Exception as ex:
    print(ex)

f integrate(F(x, y), x, n_x, b_x)
symb,a,b y n_y b_y
f_diff_symb_p diff(F(x, y), y, y, y)

(-1)^(p+1)*P(x=symb,p=p)/fact(n=p) 1/6*(y - floor(y))^3 - 1/4*(y - floor(y))^2 + 1/12*y - 1/12*floor(y)
R_p_int= integrate(1/12*(2*(y - floor(y))^3 - 3*(y - floor(y))^2 + y - floor(y))*diff(F(x, y), y, y, y), y, n_y, b_y)


<IPython.core.display.Math object>

sum_dfdx_a_bernoullis -1/2*F(x, y) + 1/12*diff(F(x, y), x)
sum_dfdx_a_bernoullis(a) -1/2*F(n_x, y) + 1/12*diff(F(n_x, y), n_x)
f 1/2*F(n_x, y) - 1/12*diff(F(n_x, y), n_x)
symb,a,b y n_y b_y
f_diff_symb_p diff(F(x, y), y, y, y)

(-1)^(p+1)*P(x=symb,p=p)/fact(n=p) 1/6*(y - floor(y))^3 - 1/4*(y - floor(y))^2 + 1/12*y - 1/12*floor(y)
R_p_sum= integrate(1/12*(2*(y - floor(y))^3 - 3*(y - floor(y))^2 + y - floor(y))*diff(F(x, y), y, y, y), y, n_y, b_y)


<IPython.core.display.Math object>

f F(x, y)
symb,a,b x n_x b_x
f_diff_symb_p diff(F(x, y), x, x, x)

(-1)^(p+1)*P(x=symb,p=p)/fact(n=p) 1/6*(x - floor(x))^3 - 1/4*(x - floor(x))^2 + 1/12*x - 1/12*floor(x)
R_px= integrate(1/12*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, y), x, x, x), x, n_x, b_x)


...

<IPython.core.display.Math object>

f integrate(1/12*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, y), x, x, x), x, n_x, b_x)
symb,a,b y n_y b_y
f_diff_symb_p 1/12*integrate((2*x - 2*floor(x) - 1)*(x - floor(x))*(x - floor(x) - 1)*diff(F(x, y), x, x, x, y, y, y), x, n_x, b_x)

(-1)^(p+1)*P(x=symb,p=p)/fact(n=p) 1/6*(y - floor(y))^3 - 1/4*(y - floor(y))^2 + 1/12*y - 1/12*floor(y)
R_p_R_p= integrate(1/144*(2*(y - floor(y))^3 - 3*(y - floor(y))^2 + y - floor(y))*integrate((2*x - 2*floor(x) - 1)*(x - floor(x))*(x - floor(x) - 1)*diff(F(x, y), x, x, x, y, y, y), x, n_x, b_x), y, n_y, b_y)


<IPython.core.display.Math object>

R_p_R_p= integrate(1/144*(2*(y - floor(y))^3 - 3*(y - floor(y))^2 + y - floor(y))*integrate((2*x - 2*floor(x) - 1)*(x - floor(x))*(x - floor(x) - 1)*diff(F(x, y), x, x, x, y, y, y), x, n_x, b_x), y, n_y, b_y)


<IPython.core.display.Math object>

In [23]:
sxy_symb = euler_maclaurin1(euler_maclaurin1(F(x,y),x,a_x,b_x,p,f_diff_symb_p=None), \
                            y,a_y,b_y,p,f_diff_symb_p=None)

In [24]:
sxy_symb

1/4*F(n_x, n_y) - 1/144*integrate((2*x - 2*floor(x) - 1)*(x - floor(x))*(x - floor(x) - 1)*diff(F(x, n_y), x, x, x, n_y), x, n_x, b_x) + integrate(1/144*(2*(y - floor(y))^3 - 3*(y - floor(y))^2 + y - floor(y))*(integrate((2*x - 2*floor(x) - 1)*(x - floor(x))*(x - floor(x) - 1)*diff(F(x, y), x, x, x, y, y, y), x, n_x, b_x) + 12*integrate(diff(F(x, y), y, y, y), x, n_x, b_x) - diff(F(n_x, y), n_x, y, y, y) + 6*diff(F(n_x, y), y, y, y)), y, n_y, b_y) + 1/24*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x), x, n_x, b_x) - 1/12*integrate(-6*F(n_x, y), y, n_y, b_y) - 1/12*integrate(-12*integrate(1/6*x^3*diff(F(x, y), x, x, x), x, n_x, b_x), y, n_y, b_y) - 1/12*integrate(-12*integrate(-1/2*x^2*floor(x)*diff(F(x, y), x, x, x), x, n_x, b_x), y, n_y, b_y) - 1/12*integrate(-12*integrate(1/2*x*floor(x)^2*diff(F(x, y), x, x, x), x, n_x, b_x), y, n_y, b_y) - 1/12*integrate(-12*integrate(-1/6*floor(x)^3*diff(F(x, y), x, x, x), x, n_x, b_x), y, n_y, b_y) - 1/

In [25]:
latex(sxy_symb)

\frac{1}{4} \, F\left(n_{x}, n_{y}\right) - \frac{1}{144} \, \int_{n_{x}}^{b_{x}} {\left(2 \, x - 2 \, \left \lfloor x \right \rfloor - 1\right)} {\left(x - \left \lfloor x \right \rfloor\right)} {\left(x - \left \lfloor x \right \rfloor - 1\right)} \frac{\partial^{4}}{(\partial x)^{3}\partial n_{y}}F\left(x, n_{y}\right)\,{d x} + \int_{n_{y}}^{b_{y}} \frac{1}{144} \, {\left(2 \, {\left(y - \left \lfloor y \right \rfloor\right)}^{3} - 3 \, {\left(y - \left \lfloor y \right \rfloor\right)}^{2} + y - \left \lfloor y \right \rfloor\right)} {\left(\int_{n_{x}}^{b_{x}} {\left(2 \, x - 2 \, \left \lfloor x \right \rfloor - 1\right)} {\left(x - \left \lfloor x \right \rfloor\right)} {\left(x - \left \lfloor x \right \rfloor - 1\right)} \frac{\partial^{6}}{(\partial x)^{3}(\partial y)^{3}}F\left(x, y\right)\,{d x} + 12 \, \int_{n_{x}}^{b_{x}} \frac{\partial^{3}}{(\partial y)^{3}}F\left(x, y\right)\,{d x} - \frac{\partial^{4}}{\partial n_{x}(\partial y)^{3}}F\left(n_{x}, y\right) + 6 \, \frac{\

In [26]:
display(Math(latex(sxy_symb)))

<IPython.core.display.Math object>

In [27]:
f(x,y)=1/(x^2)
assume(y>0)
assume(x>0)
a_x = 1
b_x = Infinity

a_y = 1
b_y = Infinity

s [_SAGE_VAR_y>0]
parser_output [y > 0]
s [_SAGE_VAR_x>0]
parser_output [x > 0]


In [28]:
calc_R_p=True
use_int_R_p_j=False
use_int_R_p_shifted_first=True
change_R_p_order=True

In [29]:
ans = my_euler_maclaurin1(f(x,y), x, a_x, b_x, p, f_diff_symb_p=None)
ans

integrate(-2*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))/x^5, x, 1, +Infinity) + 5/3

In [30]:
ans = euler_maclaurin1(f(x,y), x, a_x, b_x, p, f_diff_symb_p=None)
ans

integrate(-2*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))/x^5, x, 1, +Infinity) + 5/3

In [31]:
ans.n()

s inf
parser_output +Infinity


1.64493413560982

In [32]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=True
change_R_p_order=False

In [33]:
ans = my_euler_maclaurin1(f,x,a_x,b_x,p, f_diff_symb_p=None)
ans

s [_SAGE_VAR_j>0]
parser_output [j > 0]
s _SAGE_VAR_x-_SAGE_VAR_v
parser_output -v + x
s _SAGE_VAR_x-_SAGE_VAR_v
parser_output -v + x
s _SAGE_VAR_x-_SAGE_VAR_v
parser_output -v + x
s _SAGE_VAR_x-_SAGE_VAR_v
parser_output -v + x
s _SAGE_VAR_y-_SAGE_VAR_v
parser_output -v + y
s _SAGE_VAR_y-_SAGE_VAR_v
parser_output -v + y
s _SAGE_VAR_y-_SAGE_VAR_v
parser_output -v + y
s _SAGE_VAR_y-_SAGE_VAR_v
parser_output -v + y
maxima_eval_res ((MTIMES SIMP) ((RAT SIMP) 1 6)
 ((MPLUS SIMP) -10 ((MEXPT SIMP RATSIMP) $%PI 2)))
s pi
parser_output pi
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y


(x, y) |--> 1/6*pi^2

In [34]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None)
ans

sum(integrate(-2*(2*v_x^3 - 3*v_x^2 + v_x)/(j_x + v_x)^5, v_x, 0, 1), j_x, 1, +Infinity) + 5/3

In [35]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=True
change_R_p_order=True

In [36]:
ans = my_euler_maclaurin1(f,x,a_x,b_x,p, f_diff_symb_p=None)
ans

maxima_eval_res ((MPLUS SIMP)
 ((MTIMES SIMP)
  ((MPLUS SIMP) ((MTIMES SIMP RATSIMP) -2 ((%ZETA SIMP) 5))
   ((MTIMES SIMP RATSIMP) 2 (($GEN_HARMONIC_NUMBER SIMP) 5 |$_SAGE_VAR_v|)))
  |$_SAGE_VAR_v|)
 ((MTIMES SIMP)
  ((MPLUS SIMP) ((MTIMES SIMP RATSIMP) 6 ((%ZETA SIMP) 5))
   ((MTIMES SIMP RATSIMP) -6 (($GEN_HARMONIC_NUMBER SIMP) 5 |$_SAGE_VAR_v|)))
  ((MEXPT SIMP RATSIMP) |$_SAGE_VAR_v| 2))
 ((MTIMES SIMP)
  ((MPLUS SIMP) ((MTIMES SIMP RATSIMP) -4 ((%ZETA SIMP) 5))
   ((MTIMES SIMP RATSIMP) 4 (($GEN_HARMONIC_NUMBER SIMP) 5 |$_SAGE_VAR_v|)))
  ((MEXPT SIMP RATSIMP) |$_SAGE_VAR_v| 3)))
s zeta(5)
parser_output zeta(5)
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y


(x, y) |--> integrate(4*v^3*(harmonic_number(v, 5) - zeta(5)) - 6*v^2*(harmonic_number(v, 5) - zeta(5)) + 2*v*(harmonic_number(v, 5) - zeta(5)), v, 0, 1) + 5/3

In [37]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None)
ans

integrate(sum(-2*(2*v_x^3 - 3*v_x^2 + v_x)/(j_x + v_x)^5, j_x, 1, +Infinity), v_x, 0, 1) + 5/3

In [38]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None, hold_int=False)
ans

integrate(sum(-2*(2*v_x^3 - 3*v_x^2 + v_x)/(j_x + v_x)^5, j_x, 1, +Infinity), v_x, 0, 1) + 5/3

In [39]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=True
change_R_p_order=False

In [40]:
ans = my_euler_maclaurin1(f,x,a_x,b_x,p, f_diff_symb_p=None)
ans

s _SAGE_VAR_x-_SAGE_VAR_v
parser_output -v + x
s _SAGE_VAR_x-_SAGE_VAR_v
parser_output -v + x
s _SAGE_VAR_x-_SAGE_VAR_v
parser_output -v + x
s _SAGE_VAR_x-_SAGE_VAR_v
parser_output -v + x
s _SAGE_VAR_y-_SAGE_VAR_v
parser_output -v + y
s _SAGE_VAR_y-_SAGE_VAR_v
parser_output -v + y
s _SAGE_VAR_y-_SAGE_VAR_v
parser_output -v + y
s _SAGE_VAR_y-_SAGE_VAR_v
parser_output -v + y
maxima_eval_res ((MTIMES SIMP) ((RAT SIMP) 1 6)
 ((MPLUS SIMP) -10 ((MEXPT SIMP RATSIMP) $%PI 2)))
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_y-_SAGE_VAR_x
parser_output -x + y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y
s _SAGE_VAR_x-_SAGE_VAR_y
parser_output x - y


(x, y) |--> 1/6*pi^2

In [41]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None)
ans

sum(integrate(-2*(2*v_x^3 - 3*v_x^2 + v_x)/(j_x + v_x)^5, v_x, 0, 1), j_x, 1, +Infinity) + 5/3

In [42]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None, hold_int=False)
ans

sum(1/6*(6*j_x^2 + 15*j_x + 10)/(j_x^3 + 3*j_x^2 + 3*j_x + 1) - 1/6*(6*j_x^2 - 3*j_x + 1)/j_x^3, j_x, 1, +Infinity) + 5/3

In [43]:
f(x,y)=1/((x^2+y^2)^2)

assume(y>0)
assume(x>0)

In [44]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=False
change_R_p_order=False

In [45]:
use_int_R_p_j = False
#use_num_int = False

int_int = integral(integral           (f(x,y),(x, a_x, b_x)),  (y, a_y, b_y)) 
#print("int_int=$", latex(int_int),"$")
print("int_int=", int_int.n())
display(Math(latex(int_int)))

int_sum = integral(sum_dfdx_bernoulis (f(x,y), x, a_x, b_x,p), (y, a_y, b_y)) 
#print("int_sum=$", latex(int_sum.expand()),"$")
print("int_sum=", int_sum.n())
display(Math(latex(int_sum.expand())))

try:
    use_num_int = True
    use_int_R_p_j=True
    R_p = euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p, f_diff_symb_p=None)
    if type(R_p) is tuple:
        int_R_p = integral(R_p[0], (y, a_y, b_y), hold=True)
    else:
        int_R_p = integral(R_p, (y, a_y, b_y), hold=True)

    #print("int_R_p=$",latex(int_R_p),"$")
    print("int_R_p=", int_R_p)
    display(Math(latex(int_R_p)))
    #display(Math(latex(int_R_p.n())))
except Exception as ex:
    print(ex)

use_num_int = True
use_int_R_p_j = False

f_diff_x_p_str = str(f(x,y).diff(x,p))
print("f_diff_x_p_str=", f_diff_x_p_str)
exec(preparse("f_diff_x_p = " + f_diff_x_p_str))
#print(preparse("f_diff_x_p = " + f_diff_x_p_str))
print("f_diff_x_p = $", latex(f_diff_x_p),"$")

int_R_p_ans = integral_euler_maclaurin_R_p(f(x,y), x, a_x, b_x, y, a_y, b_y, p, f_diff_x_p=f_diff_x_p, hold_int=False)
if type(int_R_p_ans) is tuple:
    (int_R_p,int_int_R_p) = int_R_p_ans
else:
    int_R_p = int_R_p_ans

#print("int_R_p=$",latex(int_R_p),"$")
print("int_R_p=",int_R_p)
#print("int_R_p=", int_R_p.n())
display(Math(latex(int_R_p)))

int_int= 0.142699081698724


<IPython.core.display.Math object>

int_sum= 0.0861909779407098


<IPython.core.display.Math object>

int_R_p= integrate(sum(integrate(-2*(2*v_x^3 - 3*v_x^2 + v_x)*(8*(j_x + v_x)^3/((j_x + v_x)^2 + y^2)^5 - 3*(j_x + v_x)/((j_x + v_x)^2 + y^2)^4), v_x, 0, 1), j_x, 1, +Infinity), y, 1, +Infinity)


<IPython.core.display.Math object>

f_diff_x_p_str= -192*x^3/(x^2 + y^2)^5 + 72*x/(x^2 + y^2)^4
f_diff_x_p = $ -\frac{192 \, x^{3}}{{\left(x^{2} + y^{2}\right)}^{5}} + \frac{72 \, x}{{\left(x^{2} + y^{2}\right)}^{4}} $


<ipython-input-1-b7de2e928668>:5: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral = integrate.quad(f, a, b)


int_R_p= -0.00024505599786148506


<IPython.core.display.Math object>

In [46]:
int_int

1/8*pi - 1/4

In [47]:
int_int.n()

0.142699081698724

In [48]:
int_sum

3/32*pi - 5/24

In [49]:
int_sum.n()

0.0861909779407098

In [50]:
int_R_p

-0.00024505599786148506

In [51]:
try:
    print(sum(int_int_R_p, j_x, 1, +Infinity))
except Exception as ex:
    print(ex)

name 'int_int_R_p' is not defined


In [52]:
try:
    int_R_p_s = 0
    for j in range (1, 1000):
        int_R_p_term = int_int_R_p.subs(var("j_x") == j)
        int_R_p_s += int_R_p_term
        #print(j, int_R_p_term.n(), int_R_p_n.n())
    int_R_p_n = int_R_p_s.n()
    print(int_R_p_n)
except Exception as ex:
    print(ex)
    int_R_p_n = int_R_p
    print(int_R_p_n)

name 'int_int_R_p' is not defined
-0.00024505599786148506


In [53]:
logging = True
sum_int = sum_dfdx_bernoulis(integral           (f(x,y),(x, a_x, b_x)),  y, a_y, b_y, p)
#print("sum_int=$",latex(sum_int),"$")
print("sum_int=",sum_int.n())
display(Math(latex(sum_int)))

sum_sum = sum_dfdx_bernoulis(sum_dfdx_bernoulis (f(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
#print("sum_sum=$",latex(sum_sum),"$")
print("sum_sum=", sum_sum.n())
display(Math(latex(sum_sum)))

if True:
    f_diff_symb_p_str = str(f(x,y).diff(x,p))
    print("f_diff_symb_p_str=", f_diff_symb_p_str)
    exec(preparse("f_diff_symb_p = " + f_diff_symb_p_str))
    print(preparse("f_diff_symb_p = " + f_diff_symb_p_str))
    print("f_diff_symb_p = ", f_diff_symb_p)
    #print("f_diff_symb_p = $", latex(f_diff_symb_p),"$")
    display(Math(latex(f_diff_symb_p)))
    #display(Math(latex(f_diff_symb_p(x,y))))
    
    use_num_int = False

    R_px_ans = euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p, f_diff_symb_p=f_diff_symb_p(x,y))
    if type(R_px_ans) is tuple:
        R_px = R_px_ans[0]
    else:
        R_px = R_px_ans
    print("R_px=",R_px)
    #print("R_px=$",latex(R_px),"$")
    display(Math(latex(R_px)))
    
    # Sagemath 9.3 gives wrong result here due to    
    # https://trac.sagemath.org/ticket/32160
    sum_R_p = sum_dfdx_bernoulis(R_px, y, a_y, b_y, p)
    print("sumy_R_px=",sum_R_p)
    #print("sumy_R_px=$",latex(sum_R_p),"$")
    display(Math(latex(sum_R_p)))
    display(Math(latex(sum_R_p.n())))
else:
    (sum_R_p, dfdx_a_bernoullis_terms) = sum_dfdx_bernoulis_euler_maclaurin_R_p(f(x,y), x, a_x, b_x,    y, a_y, b_y, p)
    #print("sum_R_p=$",latex(sum_R_p),"$")
    #print("sum_R_p=$",sum_R_p.n())
    display(Math(latex(sum_R_p)))
    display(Math(latex(sum_R_p.n())))

sum_dfdx_a_bernoullis 1/4*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3) + 1/24*(5*y^4 + 3*y^2)*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3)^2 - 1/24*(2*y*arctan(1/y) - 1/(1/y^2 + 1) - 1/(y^2*(1/y^2 + 1)) + 1)/(y^5 + y^3) - 1/8*pi/y^3 - 1/16*pi/y^4
sum_dfdx_a_bernoullis(a) -3/32*pi + 5/24
sum_int= 0.0861909779407098


<IPython.core.display.Math object>

sum_dfdx_a_bernoullis -1/2/(x^2 + y^2)^2 - 1/3*x/(x^2 + y^2)^3
sum_dfdx_a_bernoullis(a) -1/2/(y^2 + 1)^2 - 1/3/(y^2 + 1)^3
sum_dfdx_a_bernoullis -1/4/(y^2 + 1)^2 - 1/6*y/(y^2 + 1)^3 - 1/6/(y^2 + 1)^3 - 1/6*y/(y^2 + 1)^4
sum_dfdx_a_bernoullis(a) -11/96
sum_sum= 0.114583333333333


<IPython.core.display.Math object>

f_diff_symb_p_str= -192*x^3/(x^2 + y^2)^5 + 72*x/(x^2 + y^2)^4
f_diff_symb_p = -Integer(192)*x**Integer(3)/(x**Integer(2) + y**Integer(2))**Integer(5) + Integer(72)*x/(x**Integer(2) + y**Integer(2))**Integer(4)
f_diff_symb_p =  -192*x^3/(x^2 + y^2)^5 + 72*x/(x^2 + y^2)^4


...

f (x^2 + y^2)^(-2)
symb,a,b x 1 +Infinity
f_diff_symb_p -192*x^3/(x^2 + y^2)^5 + 72*x/(x^2 + y^2)^4

(-1)^(p+1)*P(x=symb,p=p)/fact(n=p) 1/6*(x - floor(x))^3 - 1/4*(x - floor(x))^2 + 1/12*x - 1/12*floor(x)
R_px= integrate(-2*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(8*x^3/(x^2 + y^2)^5 - 3*x/(x^2 + y^2)^4), x, 1, +Infinity)


<IPython.core.display.Math object>

sum_dfdx_a_bernoullis -1/2*integrate(-2*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(8*x^3/(x^2 + y^2)^5 - 3*x/(x^2 + y^2)^4), x, 1, +Infinity) + 4/3*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(10*x^3*y/(x^2 + y^2)^6 - 3*x*y/(x^2 + y^2)^5), x, 1, +Infinity)
sum_dfdx_a_bernoullis(a) 1/32*pi + integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(8*x^3/(x^2 + 1)^5 - 3*x/(x^2 + 1)^4), x, 1, +Infinity) - 5/96
sumy_R_px= -1/32*pi - integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(8*x^3/(x^2 + 1)^5 - 3*x/(x^2 + 1)^4), x, 1, +Infinity) + 5/96


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [54]:
# https://trac.sagemath.org/ticket/32160
4/3*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(10*x^3/(x^2 + 1)^6 - 3*x/(x^2 + 1)^5), x, 1, +Infinity)

1/32*pi - 5/96

In [55]:
# https://trac.sagemath.org/ticket/32160
(4/3*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(10*x^3/(x^2 + 1)^6 - 3*x/(x^2 + 1)^5), x, 1, +Infinity)).n()

0.0460914370913477

In [56]:
4/3*num_int((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(10*x^3/(x^2 + 1)^6 - 3*x/(x^2 + 1)^5), 1, +Infinity)

0.0013260640766765055

In [57]:
# https://trac.sagemath.org/ticket/32160
(-1/2*integrate(-2*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(8*x^3/(x^2 + 1)^5 - 3*x/(x^2 + 1)^4), x, 1, +Infinity) 
+ 4/3*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(10*x^3/(x^2 + 1)^6 - 3*x/(x^2 + 1)^5), x, 1, +Infinity)).n()

0.0473559395595944

In [58]:
(-1/2*num_int(-2*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(8*x^3/(x^2 + 1)^5 - 3*x/(x^2 + 1)^4), 1, +Infinity) 
+ 4/3*num_int((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(10*x^3/(x^2 + 1)^6 - 3*x/(x^2 + 1)^5), 1, +Infinity))

0.0025904503962547384

In [59]:
sum_R_p

-1/32*pi - integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(8*x^3/(x^2 + 1)^5 - 3*x/(x^2 + 1)^4), x, 1, +Infinity) + 5/96

In [60]:
display(Math(latex(sum_R_p)))

<IPython.core.display.Math object>

In [61]:
sum_R_p_n = sum_R_p.n()
sum_R_p_n

-0.0473559395595944

In [62]:
if use_int_R_p_j:
    print(dfdx_a_bernoullis_terms(j_x))

In [63]:
if use_int_R_p_j:
    sum_R_p_j_n = 0
    for j_x in range(1,100):
        term = dfdx_a_bernoullis_terms(j_x)
        sum_R_p_j_n += term
        #print(j_x, term.n(), sum_R_p_n)
    print(sum_R_p_j_n.n())

In [64]:
use_int_R_p_j = False
use_num_int = False
R_p_int = euler_maclaurin_R_p(integral           (f(x,y),(x, a_x, b_x), hold = False),  y, a_y, b_y, p, f_diff_symb_p=None)
print("R_p_int=$",latex(R_p_int),"$")
#print("R_p_int=", R_p_int.n())
display(Math(latex(R_p_int)))

R_p_sum = euler_maclaurin_R_p(sum_dfdx_bernoulis (f(x,y), x, a_x, b_x,p), y, a_y, b_y, p, f_diff_symb_p=None, hold_int=False)
print("R_p_sum=$",latex(R_p_sum),"$")
#print("R_p_sum=$",R_p_sum.n())
display(Math(latex(R_p_sum)))


f -1/2*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3) + 1/4*pi/y^3
symb,a,b y 1 +Infinity

(-1)^(p+1)*P(x=symb,p=p)/fact(n=p) 1/6*(y - floor(y))^3 - 1/4*(y - floor(y))^2 + 1/12*y - 1/12*floor(y)
R_p_int=$ \int_{1}^{+\infty} \frac{1}{12} \, {\left(2 \, {\left(y - \left \lfloor y \right \rfloor\right)}^{3} - 3 \, {\left(y - \left \lfloor y \right \rfloor\right)}^{2} + y - \left \lfloor y \right \rfloor\right)} {\left(\frac{3 \, {\left(5 \, y^{4} + 3 \, y^{2}\right)}^{3} {\left(y^{2} \arctan\left(\frac{1}{y}\right) + y + \arctan\left(\frac{1}{y}\right)\right)}}{{\left(y^{5} + y^{3}\right)}^{4}} - \frac{6 \, {\left(5 \, y^{4} + 3 \, y^{2}\right)} {\left(10 \, y^{3} + 3 \, y\right)} {\left(y^{2} \arctan\left(\frac{1}{y}\right) + y + \arctan\left(\frac{1}{y}\right)\right)}}{{\left(y^{5} + y^{3}\right)}^{3}} + \frac{3 \, {\left(y^{2} \arctan\left(\frac{1}{y}\right) + y + \arctan\left(\frac{1}{y}\right)\right)} {\left(10 \, y^{2} + 1\right)}}{{\left(y^{5} + y^{3}\right)}^{2}} - \frac{3 \, {\l

<IPython.core.display.Math object>

sum_dfdx_a_bernoullis -1/2/(x^2 + y^2)^2 - 1/3*x/(x^2 + y^2)^3
sum_dfdx_a_bernoullis(a) -1/2/(y^2 + 1)^2 - 1/3/(y^2 + 1)^3
f 1/2/(y^2 + 1)^2 + 1/3/(y^2 + 1)^3
symb,a,b y 1 +Infinity

(-1)^(p+1)*P(x=symb,p=p)/fact(n=p) 1/6*(y - floor(y))^3 - 1/4*(y - floor(y))^2 + 1/12*y - 1/12*floor(y)
R_p_sum=$ -\frac{1}{3} \, \int_{1}^{+\infty} {\left(2 \, {\left(y - \left \lfloor y \right \rfloor\right)}^{3} - 3 \, {\left(y - \left \lfloor y \right \rfloor\right)}^{2} + y - \left \lfloor y \right \rfloor\right)} {\left(\frac{24 \, y^{3}}{{\left(y^{2} + 1\right)}^{5}} - \frac{9 \, y}{{\left(y^{2} + 1\right)}^{4}} + \frac{40 \, y^{3}}{{\left(y^{2} + 1\right)}^{6}} - \frac{12 \, y}{{\left(y^{2} + 1\right)}^{5}}\right)}\,{d y} $


<IPython.core.display.Math object>

In [65]:
R_p_int

integrate(1/12*(2*(y - floor(y))^3 - 3*(y - floor(y))^2 + y - floor(y))*(3*(5*y^4 + 3*y^2)^3*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3)^4 - 6*(5*y^4 + 3*y^2)*(10*y^3 + 3*y)*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3)^3 + 3*(y^2*arctan(1/y) + y + arctan(1/y))*(10*y^2 + 1)/(y^5 + y^3)^2 - 3*(5*y^4 + 3*y^2)^2*(2*y*arctan(1/y) - 1/(1/y^2 + 1) - 1/(y^2*(1/y^2 + 1)) + 1)/(y^5 + y^3)^3 + 3*(10*y^3 + 3*y)*(2*y*arctan(1/y) - 1/(1/y^2 + 1) - 1/(y^2*(1/y^2 + 1)) + 1)/(y^5 + y^3)^2 - 3*(5*y^4 + 3*y^2)*(1/(y*(1/y^2 + 1)) - 1/(y^3*(1/y^2 + 1)) + 1/(y^3*(1/y^2 + 1)^2) + 1/(y^5*(1/y^2 + 1)^2) - arctan(1/y))/(y^5 + y^3)^2 + (3/(y^4*(1/y^2 + 1)) - 1/(y^4*(1/y^2 + 1)^2) - 7/(y^6*(1/y^2 + 1)^2) + 4/(y^6*(1/y^2 + 1)^3) + 4/(y^8*(1/y^2 + 1)^3))/(y^5 + y^3) - 15*pi/y^6), y, 1, +Infinity)

In [66]:
integral           (f(x,y),(x, a_x, b_x), hold = False)

-1/2*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3) + 1/4*pi/y^3

In [67]:
R_p_int_n = R_p_int.n()
R_p_int_n

-0.000245074944248043

In [68]:
R_p_sum

-1/3*integrate((2*(y - floor(y))^3 - 3*(y - floor(y))^2 + y - floor(y))*(24*y^3/(y^2 + 1)^5 - 9*y/(y^2 + 1)^4 + 40*y^3/(y^2 + 1)^6 - 12*y/(y^2 + 1)^5), y, 1, +Infinity)

In [69]:
display(Math(latex(R_p_sum)))

<IPython.core.display.Math object>

In [70]:
R_p_sum_n = R_p_sum.n()
R_p_sum_n

-0.00259056586591009

In [71]:
display(Math(latex(sum_R_p)))

<IPython.core.display.Math object>

In [72]:
#use_num_int = True

#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
#R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p(f(x,y), x, a_x, b_x,    y, a_y, b_y, p)
#print("R_p_R_p=$",latex(R_p_R_p),"$")
#print("R_p_R_p=$", R_p_R_p.n())
#display(Math(latex(R_p_R_p)))

In [73]:
exec(preparse("f_diff_x_p_diff_y_p = lambda x,y:" + str(f(x,y).diff(x,p).diff(y,p))))
print("f_diff_x_p_diff_y_p", f_diff_x_p_diff_y_p(x,y))
R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p_num_int(f_diff_x_p_diff_y_p, x, a_x, b_x, y, a_y, b_y, p)

f_diff_x_p_diff_y_p 322560*x^3*y^3/(x^2 + y^2)^8 - 69120*x^3*y/(x^2 + y^2)^7 - 69120*x*y^3/(x^2 + y^2)^7 + 17280*x*y/(x^2 + y^2)^6


In [74]:
R_p_R_p

0.00038638018879801886

# Summary Euler–Maclaurin 2D for $\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}$

In [75]:
int_int, int_sum, int_R_p

(1/8*pi - 1/4, 3/32*pi - 5/24, -0.00024505599786148506)

In [76]:
sum_int, sum_sum, sum_R_p

(3/32*pi - 5/24,
 11/96,
 -1/32*pi - integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*(8*x^3/(x^2 + 1)^5 - 3*x/(x^2 + 1)^4), x, 1, +Infinity) + 5/96)

In [77]:
R_p_int, R_p_sum, R_p_R_p

(integrate(1/12*(2*(y - floor(y))^3 - 3*(y - floor(y))^2 + y - floor(y))*(3*(5*y^4 + 3*y^2)^3*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3)^4 - 6*(5*y^4 + 3*y^2)*(10*y^3 + 3*y)*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3)^3 + 3*(y^2*arctan(1/y) + y + arctan(1/y))*(10*y^2 + 1)/(y^5 + y^3)^2 - 3*(5*y^4 + 3*y^2)^2*(2*y*arctan(1/y) - 1/(1/y^2 + 1) - 1/(y^2*(1/y^2 + 1)) + 1)/(y^5 + y^3)^3 + 3*(10*y^3 + 3*y)*(2*y*arctan(1/y) - 1/(1/y^2 + 1) - 1/(y^2*(1/y^2 + 1)) + 1)/(y^5 + y^3)^2 - 3*(5*y^4 + 3*y^2)*(1/(y*(1/y^2 + 1)) - 1/(y^3*(1/y^2 + 1)) + 1/(y^3*(1/y^2 + 1)^2) + 1/(y^5*(1/y^2 + 1)^2) - arctan(1/y))/(y^5 + y^3)^2 + (3/(y^4*(1/y^2 + 1)) - 1/(y^4*(1/y^2 + 1)^2) - 7/(y^6*(1/y^2 + 1)^2) + 4/(y^6*(1/y^2 + 1)^3) + 4/(y^8*(1/y^2 + 1)^3))/(y^5 + y^3) - 15*pi/y^6), y, 1, +Infinity),
 -1/3*integrate((2*(y - floor(y))^3 - 3*(y - floor(y))^2 + y - floor(y))*(24*y^3/(y^2 + 1)^5 - 9*y/(y^2 + 1)^4 + 40*y^3/(y^2 + 1)^6 - 12*y/(y^2 + 1)^5), y, 1, +Infinity),
 0.00038638018879801886)

Numerical Euler–Maclaurin 2D for $\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}$

In [78]:
int_int.n(), int_sum.n(), int_R_p_n

(0.142699081698724, 0.0861909779407098, -0.00024505599786148506)

In [79]:
sum_int.n(), sum_sum.n(), sum_R_p_n

(0.0861909779407098, 0.114583333333333, -0.0473559395595944)

In [80]:
R_p_int_n, R_p_sum_n, R_p_R_p

(-0.000245074944248043, -0.00259056586591009, 0.00038638018879801886)

In [81]:
int_int.n() + int_sum.n() + int_R_p_n + \
sum_int.n() + sum_sum.n() + sum_R_p_n + \
R_p_int_n + R_p_sum_n + R_p_R_p

0.379614114734661

In [82]:
int_int.n() + int_sum.n() + int_R_p_n + \
sum_int.n() + sum_sum.n() + R_p_sum_n + \
R_p_int_n + R_p_sum_n + R_p_R_p

0.424379488428345

$\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S}$

In [83]:
def zeta_2D_1_1(S, N):
    
    def zeta_2D_row(S, first, N):
        #print("N", N, [m for m in range(0,N)])
        row_sum = 0
        for m in range(first, N):
            row_sum += (N^2+m^2)^(-S)
            #print("m", m, "row_sum=", row_sum)
        return row_sum

    def zeta_2D_diag(S, N):
        diag_elem = (2*N^2)^(-S)
        #print("N", N, "diag_elem=", diag_elem)
        return diag_elem
    
    zeta_2D_sum = 0
    for n in range(1,N):
        zeta_2D_add = 2 * zeta_2D_row(S, 1, n) + zeta_2D_diag(S, n)
        zeta_2D_sum += zeta_2D_add
        
        #print(n, zeta_2D_add, zeta_2D_sum)
    return zeta_2D_sum

In [84]:
zeta_2D_1_1(2, 1000)

0.4243791332035987

R_p=False (Euler–Maclaurin 2D without remainer part)

In [85]:
int_int.n() + int_sum.n() + \
sum_int.n() + sum_sum.n()

0.429664370913477

# Two Dimensional Linear Zeta Function

https://core.ac.uk/download/pdf/82162715.pdf

JOURNAL OF NUMBER THEORY 35, 62-71 (1990)

Evaluation of Multidimensional Linear Zeta Functions

ALFRED ACTOR*

Department of Physics, The Pennsylvania State University,

Fogelsville, Pennsylvania 18051

Communicated by Hans Zassenhaus

Received January 28, 1988; revised May 5, 1989

A prototypical formula of this type is [1] 

$\sum\limits_{m,n=-\infty}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S} = 4\,\zeta\left(S\right)\,\beta\left(S\right)$

where of course, $\zeta\left(S\right)$ is the Riemann $\zeta$-function [2]

while $\beta\left(S\right) = \sum\limits_{n=0}^{\infty}\left( - \right)^n \left(2n + 1\right)^{-S}$
defines, for $Res > 0$, a function whose properties throughout the s-plane are
well known [3].

Thus Eq. (1) fully reveals the properties of the twodimensional Epstein $\zeta$-function [4] on the left. 



1. G. H. HARDY, Notes on some points in the integral calculus LII, Mess. Math. 49 (1919),
85-91.
2. H. M. EDWARDS, “Riemann’s Zeta Function,” Academic Press, New York, 1974.
3. A. ERDELYI, W. MAGNUS, F. OEERHETTINGER, AND F. G. TRICOMI (Eds.), “Higher Transcendental Functions,” Vol. 1, McGraw-Hill, New York, 1953. 
LINEAR ZETA FUNCTIONS 71
4. P. EPSTEIN, Zur Theorie allgemeiner Zetafunctionen, Marh. Ann. 56 (1903), 615-645; Zur
Theorie allgemeiner Zetafunctionen, II, Math. Ann. 63 (19071, 205-216. 

$\sum\limits_{m,n=(0)1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S} = \zeta\left(S\right)\,\beta\left(S\right)$

$\sum\limits_{m,n=(0)1}^{\infty} \, \left(\sqrt{m^{2} + n^{2}}\right)^{-S} = \sum\limits_{m,n=(0)1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-\frac{S}{2}} = \zeta\left(\frac{S}{2}\right)\,\beta\left(\frac{S}{2}\right)$

In [86]:
def zeta_2D(S, N):
    
    def zeta_2D_row(S, first, N):
        #print("N", N, [m for m in range(0,N)])
        row_sum = 0
        for m in range(first, N):
            row_sum += (N^2+m^2)^(-S)
            #print("m", m, "row_sum=", row_sum)
        return row_sum

    def zeta_2D_diag(S, N):
        diag_elem = (2*N^2)^(-S)
        #print("N", N, "diag_elem=", diag_elem)
        return diag_elem
    
    zeta_2D_sum = 0
    for n in range(1,N):
        zeta_2D_add = zeta_2D_row(S, 0, n) + zeta_2D_row(S, 1, n) + zeta_2D_diag(S, n)
        zeta_2D_sum += zeta_2D_add
        
        #print(n, zeta_2D_add, zeta_2D_sum)
    return zeta_2D_sum

$\beta\left(S\right) = \sum\limits_{n=0}^{\infty}\left( - \right)^n \left(2n + 1\right)^{-S}$

In [87]:
from sage.calculus.calculus import symbolic_sum
n = var("n")
S = var("S")
beta1_sym = lambda S : symbolic_sum((-1)^n * (2*n + 1)^(-S), n, 0, Infinity)
beta1 = lambda S : sum((-1)^n * (2*n + 1)^(-S), n, 0, Infinity)

def beta1_dig(S,N):
    Sum = 0
    for n in range (N):
        Sum += (-1)^n * (2*n + 1)^(-S)
    return Sum

$\sum\limits_{m,n=0,1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S}=\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S}+\zeta(2 S)$

$\sum\limits_{m,n=0,1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}=\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}+\zeta(4)$

In [88]:
int_int.n() + int_sum.n() + int_R_p_n + \
sum_int.n() + sum_sum.n() + sum_R_p_n + \
R_p_int_n + R_p_sum_n + R_p_R_p + \
zeta(4).n()

1.46193734844580

$\frac{1}{4}\sum\limits_{m,n=-\infty}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S} = \,\zeta\left(S\right)\,\beta\left(S\right)$

In [89]:
beta1_dig(2,1000)*zeta(2).n()

1.50670280430638

In [90]:
zeta_2D(2, 1000)

1.5067023665809038